In [1]:
print("ok")

ok


In [23]:
%pip install -U langchain-community



Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [3]:
PINECONE_API_KEY="3ca654dc-2db3-4444-abe8-f5c5a6a70c23"
PINECONE_API_ENV="us-east-1"

In [4]:
# Extract pdf data
from unittest import loader


def load_pdf(data):
    directory_loader = DirectoryLoader(data, 
                                       glob="*.pdf",
                                       loader_cls=PyPDFLoader)
    
    documents = directory_loader.load()

    return documents

In [1]:
extracted_data = load_pdf("data/")

NameError: name 'load_pdf' is not defined

In [ ]:
extracted_data

NameError: name 'extracted_data' is not defined

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
# print(text_chunks)
print(len(text_chunks))


7020


##  download embedding model

In [7]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

In [9]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [10]:
query_result = embeddings.embed_query("Hello world")
print("length", len(query_result))

length 384


In [ ]:
query_result

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [13]:
from pinecone.grpc import PineconeGRPC as Pinecone, ServerlessSpec

pc = Pinecone(api_key='YOUR_API_KEY')

# Create a new index
index_name = "medical-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

index = pc.Index(index_name)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, 1.5]},
        {"id": "vec2", "values": [2.0, 1.0]},
        {"id": "vec3", "values": [0.1, 3.0]},
    ],
    namespace="ns1"
)
#upsert vectors

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, -2.5]},
        {"id": "vec2", "values": [3.0, -2.0]},
        {"id": "vec3", "values": [0.5, -1.5]},
    ],
    namespace="ns2"
)
# check for indexes
print(index.describe_index_stats())

# Returns:
# {'dimension': 2,
#  'index_fullness': 0.0,
#  'namespaces': {'ns1': {'vector_count': 3}, 'ns2': {'vector_count': 3}},
#  'total_vector_count': 6}
  

  #runs similarity search
query_results1 = index.query(
    namespace="ns1",
    vector=[1.0, 1.5],
    top_k=3,
    include_values=True
)

print(query_results1)

query_results2 = index.query(
    namespace="ns2",
    vector=[1.0,-2.5],
    top_k=3,
    include_values=True
)

print(query_results2)

# Returns:
# {'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, 1.5]},
#              {'id': 'vec2', 'score': 0.868243158, 'values': [2.0, 1.0]},
#              {'id': 'vec3', 'score': 0.850068152, 'values': [0.1, 3.0]}],
#  'namespace': 'ns1',
#  'usage': {'read_units': 6}}
# {'matches': [{'id': 'vec1', 'score': 1.0, 'values': [1.0, -2.5]},
#              {'id': 'vec3', 'score': 0.998274386, 'values': [0.5, -1.5]},
#              {'id': 'vec2', 'score': 0.824041963, 'values': [3.0, -2.0]}],
#  'namespace': 'ns2',
#  'usage': {'read_units': 6}}




ImportError: cannot import name 'ServerlessSpec' from 'pinecone.grpc' (c:\Users\MIKE\miniconda3\envs\mchatbot\lib\site-packages\pinecone\grpc\__init__.py)

In [20]:
%pip install pinecone
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.



In [19]:
import os
from pinecone import Pinecone

os.environ['PINECONE_API_KEY'] = '83655f48-c25b-4286-a2f1-146da0924de9'

pinecone_instance = Pinecone(api_key='83655f48-c25b-4286-a2f1-146da0924de9', environment='gcp-starter') #I dont think this line does anything but sijaitoa just incase

index_name = "medical-chatbot"

# Create a PineconeVectorStore instance with the Pinecone instance and index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

ModuleNotFoundError: No module named 'langchain_pinecone'